In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

In [2]:
os.chdir(sys.path[0]+"/../data")

In [3]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm

categories = [
  "100 metres, Men",
  "200 metres, Men",
  "400 metres, Men",
  "800 metres, Men",
  "1,500 metres, Men",
  "5,000 metres, Men",
  "10,000 metres, Men",
  "Marathon, Men",
  "110 metres Hurdles, Men",
  "400 metres Hurdles, Men",
  "3,000 metres Steeplechase, Men",
  "4 x 100 metres Relay, Men",
  "4 x 400 metres Relay, Men",
  "20 kilometres Walk, Men",
  "50 kilometres Walk, Men",
  "100 metres, Women",
  "200 metres, Women",
  "400 metres, Women",
  "800 metres, Women",
  "1,500 metres, Women",
  "5,000 metres, Women",
  "10,000 metres, Women",
  "Marathon, Women",
  "110 metres Hurdles, Women",
  "400 metres Hurdles, Women",
  "3,000 metres Steeplechase, Women",
  "4 x 100 metres Relay, Women",
  "4 x 400 metres Relay, Women",
  "20 kilometres Walk, Women",
]

data = []
for edition in tqdm(range(1,62)): # Data from 1896 to 2020
  edition_url = f"http://www.olympedia.org/editions/{edition}/sports/ATH"
  response = urllib.request.urlopen(edition_url)
  edition_soup = BeautifulSoup(response, 'html.parser')
  title = edition_soup.find_all("h1")[0]
  if "Winter" in title.text: continue # Skip winter olympics
  try:
    edition_year = int(re.findall(r"\d+", title.text)[0])
  except IndexError:
    continue # Sometimes the page seems to not exist?

  for category in categories:
    try:
      elem = edition_soup.find_all("a", string=category)[0]
    except IndexError:
      continue
    href = elem.get('href')
    event_url = "http://www.olympedia.org" + href
    response = urllib.request.urlopen(event_url)
    soup = BeautifulSoup(response, 'html.parser')
    table = soup.find_all("table", {"class":"table table-striped"})[0]
    df = pd.read_html(str(table))[0]
    try:
      # gold_medal_time = float(re.findall(r"[+-]?([0-9]+([.][0-9]*)?|[.][0-9]+)",
      #                                   df['Final'][0].split()[0])[0][0]
      #                         )
      final_time_raw = df['Final'][0].split()[0]
      h, m, s = re.findall(r"^(?:(\d{0,2})-)?(?:(\d{0,2}):)?(\d{0,2}\.?\d*)", 
                           final_time_raw)[0]
      h,m,s = int(h) if len(h)>0 else 0, int(m) if len(m)>0 else 0, float(s)
      gold_medal_time = h*60*60+m*60+s
    except KeyError:
      continue
    data.append({
        "category" : category,
        "year": edition_year,
        "time" : gold_medal_time,
        "reference": event_url,
    })

df = pd.DataFrame(data)
df.to_csv('olympics_athletic_gold_medal_times.csv')
df

100%|██████████| 61/61 [12:44<00:00, 12.53s/it]


,category,year,time,reference
0,"100 metres, Men",1896,12.00,http://www.olympedia.org/results/56000
1,"400 metres, Men",1896,54.20,http://www.olympedia.org/results/56006
2,"800 metres, Men",1896,131.00,http://www.olympedia.org/results/56011
3,"110 metres Hurdles, Men",1896,17.60,http://www.olympedia.org/results/56018
4,"100 metres, Men",1900,11.00,http://www.olympedia.org/results/56034
...,...,...,...,...
425,"5,000 metres, Women",2016,866.17,http://www.olympedia.org/results/359060
426,"400 metres Hurdles, Women",2016,53.13,http://www.olympedia.org/results/359080
427,"3,000 metres Steeplechase, Women",2016,539.75,http://www.olympedia.org/results/359093
428,"4 x 100 metres Relay, Women",2016,41.01,http://www.olympedia.org/results/359099
